In [25]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, to_date, date_format, to_timestamp, substring
os.chdir('/Users/chkapsalis/Documents/GitHub/Big_Data_Architectures/Assignments/my_assignment_3')

# For some reason i need to run this every time in order to get it work
import os
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11/libexec/openjdk.jdk/Contents/Home" 

# the .config("option", "value") arguments allow us to perform refined file I/O
spark = SparkSession.builder.master("local[1]") \
            .appName("app") \
            .config("option", "value") \
            .getOrCreate()

In [20]:
cols = 'DATETIME,LAT,LONG,DEPTH,MAGNITUDE'.split(',')

In [21]:
quakes = spark.read.csv('file:///' + os.getcwd() + '/earthquakes.csv', header=True)
quakes.printSchema()
quakes.show()

root
 |-- DATETIME: string (nullable = true)
 |-- LAT: string (nullable = true)
 |-- LONG: string (nullable = true)
 |-- DEPTH: string (nullable = true)
 |-- MAGNITUDE: string (nullable = true)

+----------------+-----+-----+-----+---------+
|        DATETIME|  LAT| LONG|DEPTH|MAGNITUDE|
+----------------+-----+-----+-----+---------+
|01/07/1965 10:22| 36.5| 26.5|   10|      5.3|
|01/10/1965 08:02|39.25|22.25|   10|      4.9|
|01/12/1965 17:26|   37|   22|   10|        4|
| 1/15/1965 14:56|36.75|21.75|   10|      4.5|
|03/09/1965 19:16|   39|   24|   10|      4.2|
|03/09/1965 20:14|   39|   24|   10|      4.4|
|03/10/1965 00:04|   39|   24|   10|      4.8|
|03/10/1965 21:50|   39|   24|   10|      4.8|
| 3/31/1965 12:01| 38.5|21.75|   10|      5.1|
|04/03/1965 05:19|38.11| 23.7|   10|      4.6|
| 5/15/1965 13:34| 37.5|21.75|   10|        4|
|07/06/1965 06:22| 38.4| 22.3|   10|      4.1|
|07/07/1965 23:26| 38.4| 22.2|   10|      3.8|
| 7/16/1965 13:54| 38.5|23.25|   10|      3.3|
|  8/1

In [ ]:
# Question 1

# This implementation is problematic due to how spark+java handle very old dates - i get NULL values due to
# overflow issues when i try to print out dates older than 1970
quakes_parsed = quakes.withColumn("parsed_dt", to_timestamp("DATETIME", "dd/MM/yyyy HH:mm"))
quakes_parsed \
    .orderBy("MAGNITUDE", ascending=False) \
      .select(
          to_date(col("parsed_dt")).alias("DATE"),
          date_format(col("parsed_dt"), "HH:mm").alias("TIME"),
          col("MAGNITUDE")
      ) \
      .limit(5) \
      .show()


+----------+-----+---------+
|      DATE| TIME|MAGNITUDE|
+----------+-----+---------+
|      NULL| NULL|      6.7|
|      NULL| NULL|      6.7|
|1983-06-08|15:43|      6.6|
|2008-08-06|12:25|      6.5|
|      NULL| NULL|      6.4|
+----------+-----+---------+



In [27]:
quakes.select(
    substring("DATETIME", 1, 10).alias("DATE"),
    substring("DATETIME", 12, 5).alias("TIME"),
    col("MAGNITUDE")
).orderBy(col("MAGNITUDE").desc()) \
 .limit(10) \
.show()


+----------+-----+---------+
|      DATE| TIME|MAGNITUDE|
+----------+-----+---------+
|2/19/1968 | 2:45|      6.7|
|10/30/2020|11:51|      6.7|
|08/06/1983|15:43|      6.6|
|06/08/2008|12:25|      6.5|
|1/18/1982 | 9:27|      6.4|
|01/08/2006|11:34|      6.4|
|2/24/1981 | 0:53|      6.3|
|12/19/1981|14:10|      6.3|
|5/24/2014 |  :25|      6.3|
|7/15/2008 |  :26|      6.2|
+----------+-----+---------+



In [ ]:
# Question 2
# Adding a properly parsed datetime column to the current dataframe
quakes_with_year = quakes.withColumn("parsed_dt", to_timestamp("DATETIME", "dd/MM/yyyy HH:mm"))

quakes_with_year.select(
            date_format(col("parsed_dt"), "yyyy").alias("YEAR")
        ) \
        .groupBy("YEAR") \
        .count() \
        .sort("YEAR", ascending=True) \
        .filter(col("YEAR") > 2010) \
        .show()


+----+-----+
|YEAR|count|
+----+-----+
|2011| 5624|
|2012| 7036|
|2013| 8056|
|2014|10172|
|2015| 6989|
|2016| 5523|
|2017| 4468|
|2018| 3431|
|2019| 5185|
|2020| 4178|
|2021| 9042|
+----+-----+



In [40]:
# Question 3

quakes_with_year.registerTempTable('quakes')

/Users/chkapsalis/.pyenv/versions/base/lib/python3.10/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [53]:
r3 = spark.sql("""
   SELECT 
      YEAR(parsed_dt),
      MIN(MAGNITUDE) AS min_magn,
      MAX(MAGNITUDE) AS max_magn,
      AVG(MAGNITUDE) AS avg_magn
   FROM quakes
   WHERE YEAR(parsed_dt) BETWEEN 2010 AND 2020
   GROUP BY YEAR(parsed_dt)
   ORDER BY YEAR(parsed_dt) ASC
    """)

In [54]:
r3.show()

+---------------+--------+--------+------------------+
|year(parsed_dt)|min_magn|max_magn|          avg_magn|
+---------------+--------+--------+------------------+
|           2010|     0.8|     5.1|2.7314732142857276|
|           2011|     0.2|     6.2|1.7438477951635865|
|           2012|     0.2|       5| 1.625028425241614|
|           2013|     0.2|     5.8|1.7396350546176786|
|           2014|     0.2|     5.7|1.8499508454581273|
|           2015|     0.3|     5.3|1.7466304192302136|
|           2016|     0.1|     4.9| 1.837624479449575|
|           2017|     0.2|     6.1|1.8582363473590011|
|           2018|     0.1|     4.8|1.9132905858350295|
|           2019|     0.5|     5.4|1.9195178399228594|
|           2020|     0.4|     5.2|1.9605313547151737|
+---------------+--------+--------+------------------+



In [58]:
# Question 4
r4 = spark.sql("""
    SELECT 
        DATETIME,
        MAGNITUDE
    FROM quakes
    WHERE (LAT BETWEEN 37.5 AND 39.0) AND (LONG BETWEEN 23.35 AND 23.55)
    ORDER BY MAGNITUDE DESC
    LIMIT 5
""")

In [59]:
r4.show()

+----------------+---------+
|        DATETIME|MAGNITUDE|
+----------------+---------+
|06/09/2015 01:09|      5.3|
|11/17/2014 23:05|      5.2|
|11/17/2014 23:09|      5.2|
| 7/19/2019 11:13|      5.1|
| 10/14/2008 2:09|      4.7|
+----------------+---------+

